# OptStab Plots of Search Results

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from OptStab import M, beta, iV, iV_a, iV_b, iV_c, iY
from scipy.interpolate import interp1d

NameError: name 'ModuleNotFoundError' is not defined

In [ ]:
VERSION = 1
from os.path import expanduser
home = expanduser("~")
FIGURE_DIR = os.path.join(home,'Scratch','OptStabFigures')
PRESENTATION = False
OutDir = os.path.join(os.path.pardir,os.path.pardir,'data','results')

In [ ]:
%matplotlib inline

In [ ]:
if PRESENTATION:
    mycolors = [(51,102,204), (153,0,0)]  
else:
    mycolors = [3*(0,), 3*(170,)]  

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(mycolors)):    
    r, g, b = mycolors[i]    
    mycolors[i] = (r / 255., g / 255., b / 255.)    

In [ ]:
with np.load(os.path.join(OutDir,'SearchResults_v'+str(VERSION)+'.npz')) as X:
    OptCyc = X['OptCyc']
    OptSS = X['OptSS']

# check if the figure directory exists and create it if it doesn't
if not os.path.isdir(FIGURE_DIR):
    os.makedirs(FIGURE_DIR)

with np.load(os.path.join(OutDir,'WelfareGrid_v'+str(VERSION)+'.npz')) as X:
    b_grid, tau_grid = X['b_grid'], X['tau_grid']
    WelfareCyc, WelfareNoCyc = X['WelfareCyc'], X['WelfareNoCyc']
    StSt, StDevs = X['SteadyState'], X['StDevs']

def myreshape(x):
    return np.reshape(x,b_grid.shape)


In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
# ax.spines["top"].set_visible(False)    
# ax.spines["bottom"].set_visible(False)    
# ax.spines["right"].set_visible(False)    
# ax.spines["left"].set_visible(False)

# Ensure that the axis ticks only show up on the bottom and left of the plot.    
# Ticks on the right and top of the plot are generally unnecessary chartjunk.    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()  

# Make sure your axis ticks are large enough to be easily read.    
# You don't want your viewers squinting to read your plot.    
plt.xticks(np.linspace(0.75, 0.9, 4), [str(x)  for x in np.linspace(0.75, 0.9, 4)], fontsize=14)    
plt.yticks(np.arange(tau_grid.min(), tau_grid.max(),0.02), [str(x)  for x in np.arange(tau_grid.min(), tau_grid.max(),0.02)], fontsize=14)    



plt.contourf(b_grid,tau_grid,myreshape(WelfareNoCyc[:,3]),20,cmap = plt.cm.gray)
plt.plot(OptSS[0],OptSS[1],'ko')

plt.xlabel('b',fontsize =14)
plt.ylabel('tau',fontsize =14)

fig.savefig(os.path.join(FIGURE_DIR,'OptimalPolicyContoursNoCycles.png'),bbox_inches = 'tight')

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
# ax.spines["top"].set_visible(False)    
# ax.spines["bottom"].set_visible(False)    
# ax.spines["right"].set_visible(False)    
# ax.spines["left"].set_visible(False)

# Ensure that the axis ticks only show up on the bottom and left of the plot.    
# Ticks on the right and top of the plot are generally unnecessary chartjunk.    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()  

# Make sure your axis ticks are large enough to be easily read.    
# You don't want your viewers squinting to read your plot.    
plt.xticks(np.linspace(0.75, 0.9, 4), [str(x)  for x in np.linspace(0.75, 0.9, 4)], fontsize=14)    
plt.yticks(np.arange(tau_grid.min(), tau_grid.max(),0.02), [str(x)  for x in np.arange(tau_grid.min(), tau_grid.max(),0.02)], fontsize=14)    


plt.contourf(b_grid,tau_grid,myreshape(WelfareCyc[:,3]),20,cmap = plt.cm.gray)
plt.plot(OptSS[0],OptSS[1],'ko')
plt.plot(OptCyc[0],OptCyc[1],'ro')
plt.xlabel('b',fontsize =14)
plt.ylabel('tau',fontsize =14)
fig.savefig(os.path.join(FIGURE_DIR,'OptimalPolicyContours.png'),bbox_inches = 'tight')

We can convert b into pre-tax replacement rates based on a two-worker household.  This conversion involves for solving for x in $$(x/2 + 1/2)^{1-\tau} = b.$$  
Which gives us
$$x = 2b^{1/(1-\tau)} - 1.$$  

In [ ]:
def RepRate(b,tau):
    return 2 * b **(1.0/(1-tau)) -1

print '-----optimal b-----'
print 'with cycles = {0:.{1}f}'.format(OptCyc[0],3)
print 'without cycles = {0:.{1}f}'.format(OptSS[0],3)
print('')

print '-----optimal tau----'
print 'with cycles = {0:.{1}f}'.format(OptCyc[1],3)
print 'without cycles = {0:.{1}f}'.format(OptSS[1],3)
print('')

print 'pre-tax UI replacement rates with a two-earner household'
print 'with cycles = {0:.{1}f}'.format(RepRate(OptCyc[0],OptCyc[1]),2)
print 'without cycles = {0:.{1}f}'.format(RepRate(OptSS[0],OptSS[1]),2)


In [ ]:
def InnerPlot(ax,X,Y,xlabel,xlims,xticks,ylims=None, yticks = None, VertLines = None, title=None, Showxticklables = True):
    ax.get_xaxis().tick_bottom()    
    ax.get_yaxis().tick_left()  

    # Shift the axis labels away from the axis
    ax.tick_params(axis='x', pad=8)
    ax.tick_params(axis='y', pad=8)

    if xticks is None:
        plt.xticks(fontsize=14,family='sans-serif')
    else:
        plt.xticks(xticks, [str(x)  for x in xticks], fontsize=14,family='sans-serif')    

    if yticks is None:
        plt.yticks(fontsize=14,family='sans-serif')
    else:
        plt.yticks(yticks, [str(x)  for x in yticks], fontsize=14,family='sans-serif')    

    if xlims is not None:
        plt.xlim(xlims)

    if ylims is not None:
        plt.ylim(ylims)


    for j in range(len(Y)):
        plt.plot(X,Y[j],lw = 2.5, color = mycolors[j])

    if title is not None:
        plt.title(title,fontsize = 14,family='serif')

#         for y in ax.get_yticks():    
#             plt.plot(ax.get_xlim(), [y] * 2, "--", lw=0.5, color="black", alpha=0.3)   

    if xlabel is not None:
        plt.xlabel(xlabel,fontsize = 14,family='serif')
    if not Showxticklables:
        # hide the x tick labels 
        ax.set_xticklabels('',visible=False)

    if VertLines is not None:
        ylimsasset = ax.get_ylim()
        for v in VertLines:
            plt.plot( [v] * 2, ylimsasset, "--", lw=0.5, color="black", alpha=0.3)
        plt.ylim(ylimsasset)
        



def Make1Plot(X,Y,xlabel,xlims,xticks,ylims=None, yticks = None, VertLines = None, title = None):
    fig = plt.figure(figsize=(5, 3))
    ax = plt.subplot(1,1,1)
    Showxticklables = True
    InnerPlot(ax,X,Y,xlabel,xlims,xticks,ylims, yticks, VertLines, title, Showxticklables)

    return fig

def Make4Plots(X,Y,xlabel,title=None,xticks = None,xlims = None,
               yticks = None,ylims = None,VertLines = None):
    
    if xticks is None:
        xticks = 4*[None,]
    if xlims is None:
        xlims = 4*[None,]
    if yticks is None:
        yticks = 4*[None,]
    if ylims is None:
        ylims = 4*[None,]
    if title is None:
        title = 4*[None,]

        
    fig = plt.figure(figsize=(10, 7.5))
    plt.subplots_adjust(hspace = 0.2, wspace = 0.4)

    for i in range(4):
        ax = plt.subplot(2,2,i+1)
        if i < 2:
            xlabel_arg = None
            Showxticklables = False
        else:
            xlabel_arg = xlabel
            Showxticklables = True
                
        
        InnerPlot(ax,X,Y[i],xlabel_arg,xlims[i],xticks[i],ylims[i], yticks[i], VertLines, title[i],Showxticklables)
        
    return fig

In [ ]:
def ConsEquiv(x):
    return np.exp((1-beta)*x)-1

In [ ]:
select_tau = 3

M.Set_b_tau(OptSS[0],tau_grid[select_tau,0])
V = M.SteadyState()[[iV_a,iV_b,iV_c,iV]]

titles = ['Skill', 'Unemployment', 'Representative agent', 'Total']

X = b_grid[select_tau]
Y = []
for i in range(4):
    cyc = ConsEquiv(myreshape(WelfareCyc[:,i])[select_tau] - V[i])
    ss = ConsEquiv(myreshape(WelfareNoCyc[:,i])[select_tau] - V[i])
    Y.append([cyc,ss])

xlabel = r'UI generosity ($b$)'
xlims = 4*[(b_grid[select_tau,0],b_grid[select_tau,-1])]  
xticks = 4*[np.linspace(0.75, 0.9, 4)]
ylims = [(-0.04,0.002), (-0.0005,0.0015), (-0.01,0.001), (-0.04,0.002)]

fig = Make4Plots(X,Y,xlabel,titles,xticks,xlims, None, None,(OptSS[0],OptCyc[0]) )

print 'tau = {0}'.format(tau_grid[select_tau,0])
fig.savefig(os.path.join(FIGURE_DIR,'WelfareGrid_b.png'),bbox_inches = 'tight')

In [ ]:
# Plot Welfare loss of sub-optimal policy
fig = plt.figure(figsize=(5, 3))
ax = plt.subplot(1,1,1)
plt.plot(X,Y[3][0] - np.max(Y[3][0]))
plt.xlabel(r'UI generosity ($b$)')
fig.savefig(os.path.join(FIGURE_DIR,'WelfareTotal_b.png'),bbox_inches = 'tight')
ylimsasset = ax.get_ylim()
for v in (OptSS[0],OptCyc[0]):
    plt.plot( [v] * 2, ylimsasset, "--", lw=0.5, color="black", alpha=0.3)
plt.ylim(ylimsasset)

In [ ]:
np.min(Y[3][0])

In [ ]:

X = b_grid[select_tau]
Y = [myreshape(StSt[:,0])[select_tau]]

xlabel = r'UI generosity ($b$)'
xlims = (b_grid[select_tau,0],b_grid[select_tau,-1])
xticks = np.linspace(0.75, 0.9, 4)

ylims = (0.055,0.075)
yticks = (0.055,0.06,0.065,0.070,0.075)

print 'SteadyState unemployment rate'

fig = Make1Plot(X,Y,xlabel,xlims,xticks,ylims=ylims,yticks = yticks,VertLines=(OptSS[0],OptCyc[0]))
fig.savefig(os.path.join(FIGURE_DIR,'SteadyState_u_vs_b.png'),bbox_inches = 'tight')



In [ ]:
titles = ['Log output', 'Unemployment', 'Hours', 'Inflation']
indices = [2,0,4,3]


X = b_grid[select_tau]
Y = []
for i in range(4):
    m = myreshape(StSt[:,indices[i]])[select_tau]
    Y.append([m])

xlabel = r'UI generosity ($b$)'
xlims = 4*[(b_grid[select_tau,0],b_grid[select_tau,-1])]  
xticks = 4*[np.linspace(0.75, 0.9, 4)]

ylims = [(-0.25,-0.23), (0.062,0.067), (0.84,0.85), (0.99,1.01)]

fig = Make4Plots(X,Y,xlabel,titles,xticks,xlims,None,None,(OptSS[0],OptCyc[0]))
print 'SteadyState'
fig.savefig(os.path.join(FIGURE_DIR,'SteadyState_b.png'),bbox_inches = 'tight')

In [ ]:
titles = ['Log output', 'Unemployment', 'Hours', 'Inflation']
indices = [2,0,4,3]


X = b_grid[select_tau]
Y = []
for i in range(4):
    m = myreshape(StDevs[:,indices[i]])[select_tau]
    Y.append([m])

xlabel = r'UI generosity ($b$)'
xlims = 4*[(b_grid[select_tau,0],b_grid[select_tau,-1])]  
xticks = 4*[np.linspace(0.75, 0.9, 4)]

yticks = [(0.022,0.024,0.026,0.028), (0.016,0.018,0.020,0.022,0.024), (0.005,0.006,0.007,0.008,0.009),(0.005,0.006,0.007,0.008,0.009)]
ylims = [(z[0],z[-1]) for z in yticks]

fig = Make4Plots(X,Y,xlabel,titles,xticks,xlims, None,None,(OptSS[0],OptCyc[0]) )
print 'Standard deviations'
fig.savefig(os.path.join(FIGURE_DIR,'StandardDeviations_b.png'),bbox_inches = 'tight')

figsd = fig

fig = Make1Plot(X,Y[0],xlabel,xlims[0],xticks[0],ylims=ylims[0],yticks = yticks[0],VertLines=[OptCyc[0],OptSS[0]])
plt.ylabel('Unemployment rate',FontSize=14)
fig.savefig(os.path.join(FIGURE_DIR,'StandardDeviation_output_vs_b.png'),bbox_inches = 'tight')



In [ ]:
# plot of Std M
X = b_grid[select_tau]
Y = myreshape(StDevs[:,5])[select_tau]
Y = Y/interp1d(X,Y)(OptCyc[0])
# Make1Plot(X,Y,xlabel,xlims,xticks,ylims=None, yticks = None, VertLines = None, title = None)
fig = Make1Plot(X,[Y,],xlabel,xlims[0],xticks[0],yticks = None,VertLines=[OptCyc[0],OptSS[0]])
plt.ylabel('Ratio of st. deviations',fontsize =14)
fig.savefig(os.path.join(FIGURE_DIR,'StandardDeviation_M_vs_b.png'),bbox_inches = 'tight')

In [ ]:
select_b = 4 
titles = ['Skill', 'Unemployment', 'Representative agent', 'Total']

M.Set_b_tau(b_grid[0,select_b],OptSS[1])
V = M.SteadyState()[[iV_a,iV_b,iV_c,iV]]

X = tau_grid[:,select_b]
Y = []
for i in range(4):
    cyc = ConsEquiv(myreshape(WelfareCyc[:,i])[:,select_b]-V[i])
    ss = ConsEquiv(myreshape(WelfareNoCyc[:,i])[:,select_b]-V[i])
    Y.append([cyc,ss])

xlabel = r'Tax progressivity ($\tau$)'
xlims = 4*[(tau_grid[0,0],tau_grid[-1,0])]  
xticks = 4*[np.arange(tau_grid.min(), tau_grid.max(), 0.03)]

fig = Make4Plots(X,Y,xlabel,titles,xticks,xlims, None, None , (OptSS[1],OptCyc[1]) )
print 'b = {0}'.format(b_grid[0,select_b])
fig.savefig(os.path.join(FIGURE_DIR,'WelfareGrid_tau.png'),bbox_inches = 'tight')



In [ ]:
# Plot Welfare loss of sub-optimal policy
fig = plt.figure(figsize=(5, 3))
ax = plt.subplot(1,1,1)
plt.plot(X,Y[3][0] - np.max(Y[3][0]))
plt.xlabel(r'Tax progressivity ($\tau$)')
fig.savefig(os.path.join(FIGURE_DIR,'WelfareTotal_tau.png'),bbox_inches = 'tight')
ylimsasset = ax.get_ylim()
for v in (OptSS[1],OptCyc[1]):
    plt.plot( [v] * 2, ylimsasset, "--", lw=0.5, color="black", alpha=0.3)
plt.ylim(ylimsasset)

In [ ]:
titles = ['Log output', 'Unemployment', 'Hours', 'Inflation']
indices = [2,0,4,3]



X = tau_grid[:,select_b]
Y = []
for i in range(4):
    m = myreshape(StSt[:,indices[i]])[:,select_b]
    Y.append([m])

xlabel = r'Tax progressivity ($\tau$)'

xlims = 4*[(tau_grid[0,0],tau_grid[-1,0])]  
xticks = 4*[np.arange(tau_grid.min(), tau_grid.max(), 0.02)]

yticks = [np.arange(-0.27,-0.21+0.001,0.02), np.arange(0.063,0.0691,0.002), np.arange(0.8,0.9001,0.02),np.arange(0.98,1.03,0.01)]
ylims = [(-0.31,-0.23), (0.0615,0.0635), (0.78,0.86), (0.98,1.02)]


fig = Make4Plots(X,Y,xlabel,titles,xticks,xlims, None,None,(OptSS[1],OptCyc[1]))

print 'SteadyState'
fig.savefig(os.path.join(FIGURE_DIR,'SteadyState_tau.png'),bbox_inches = 'tight')


M.Set_b_tau(b_grid[0,select_b],OptSS[1])
Ystst = M.SteadyState()[[iY]]

fig = Make1Plot(X,Y[0]-np.log(Ystst),xlabel,xlims[0],xticks[0])
fig.savefig(os.path.join(FIGURE_DIR,'SteadyState_output_vs_tau.png'),bbox_inches = 'tight')



In [ ]:
titles = ['Log output', 'Unemployment', 'Hours', 'Inflation']
indices = [2,0,4,3]



X = tau_grid[:,select_b]
Y = []
for i in range(4):
    m = myreshape(StDevs[:,indices[i]])[:,select_b]
    Y.append([m])

xlabel = r'Tax progressivity ($\tau$)'

xlims = 4*[(tau_grid[0,0],tau_grid[-1,0])]  
xticks = 4*[np.arange(tau_grid.min(), tau_grid.max(), 0.02)]

#ylims = [(0.01379,0.0139), (0.0111,0.0113), (0.0054,0.0059), (0.00548,0.0055)]
#yticks = [(0.014,0.016,0.018), (0.01,0.012,0.014,0.016), (0.005,0.006,0.007,0.008),(0.005,0.006,0.007,0.008)]

ylims = []
yticks = []
for i in range(4):
    ylims.append(figsd.axes[i].get_ylim())
    yticks.append(figsd.axes[i].get_yticks())

fig = Make4Plots(X,Y,xlabel,titles,xticks,xlims, yticks,ylims,(OptSS[1],OptCyc[1]))
print 'Standard deviations'
fig.savefig(os.path.join(FIGURE_DIR,'StandardDeviations_tau.png'),bbox_inches = 'tight')



fig = Make1Plot(X,Y[0],xlabel,xlims[0],xticks[0], ylims[0], yticks[0])
fig.savefig(os.path.join(FIGURE_DIR,'StandardDeviation_output_vs_tau.png'),bbox_inches = 'tight')



In [ ]:
# plot of Std M
X = tau_grid[:,select_b]
Y = myreshape(StDevs[:,5])[:,select_b]
Y = Y/Y[-1]
fig = Make1Plot(X,[Y,],xlabel,xlims[0],xticks[0])

In [ ]:
X = tau_grid[:,select_b]
Y = myreshape(StDevs[:,0])[:,select_b]
YSS = myreshape(StSt[:,0])[:,select_b]
Y = (1.0-Y)/(1.0-YSS)
fig = Make1Plot(X,[Y,],xlabel,xlims[0],xticks[0])

In [ ]:
select_b = 1 
print 'b = {0}'.format(b_grid[0,select_b])

In [ ]:
titles = ['Skill', 'Unemployment', 'Representative agent', 'Total']

M.Set_b_tau(b_grid[0,select_b],OptSS[1])
V = M.SteadyState()[[iV_a,iV_b,iV_c,iV]]

X = tau_grid[:,select_b]
Y = []
i = 0
Y.append([ConsEquiv(myreshape(WelfareCyc[:,i])[:,1]-V[i]),ConsEquiv(myreshape(WelfareCyc[:,i])[:,4]-V[i]),
             ConsEquiv(myreshape(WelfareNoCyc[:,i])[:,1]-V[i]),ConsEquiv(myreshape(WelfareNoCyc[:,i])[:,4]-V[i])])
plt.plot(X,np.array(Y).T.squeeze())
plt.legend(['Cyc 75', 'Cyc 81', 'SS 75', 'SS 81'])